In [1]:
import pandas as pd
import numpy as np

In [2]:
pheno = 'blood'
bag_type = 'bag'
if bag_type == 'bag3':
    df1 = pd.read_csv(f'results_bag3/mr_res_with_fdr.csv')
    df2 = pd.read_csv(f'results_bagm3/mr_res_with_fdr.csv')
    df1 = df1[df1['eqtl_type'] == f'{pheno} eqtls']
    df2 = df2[df2['eqtl_type'] == f'{pheno} eqtls']
    df = pd.concat([df1, df2], axis=0)
else:
    df = pd.read_csv(f'results_{bag_type}/mr_res_with_fdr.csv')
    df = df[df['eqtl_type'] == f'{pheno} eqtls']
    # replace nan with 0.999
    df['fdr'].fillna(0.999, inplace=True)
    
print(len(pd.unique(df['ensembl_gene_id'])))

55


In [3]:
df.drop_duplicates(['ensembl_gene_id', 'id.outcome', 'method'], inplace=True)
print(len(pd.unique(df['ensembl_gene_id'])))

55


In [4]:
df_phenos = pd.read_csv('data/pheno_gwas_ids.csv')
dfgenes = pd.DataFrame({'gene': pd.unique(df['hgnc_names'])})
dfgenes['x'] = 1
df_phenos['x'] = 1
df_res = pd.merge(dfgenes, df_phenos[['pheno', 'x']], on='x')

In [5]:
df_ivw = df[df['method'].isin(['Inverse variance weighted', 'Wald ratio'])].copy()
# cut min p-value 1e-9 for plotting
df_ivw['logp'] = -np.log10(df_ivw['pval'] + 1e-9)
df_ivw['log_fdrp'] = -np.log10(df_ivw['fdr'] + 1e-9)
# effect direction
df_ivw['dir'] = np.sign(df_ivw['b'])

# remove results with horizontal piet
df_ivw = df_ivw[df_ivw['egger_pval'] > 0.05]
conuts = df_ivw[df_ivw['pval'] < 0.05]['id.outcome'].value_counts()
print(len(pd.unique(df_ivw['ensembl_gene_id'])))

55


In [6]:
df_ivw['heat_value'] = df_ivw['logp'] * df_ivw['dir']
df_res_normp = pd.merge(df_res, df_ivw[['id.exposure', 'id.outcome', 'heat_value']], left_on=['gene', 'pheno'],
                  right_on=['id.exposure', 'id.outcome'], how='left')
df_res_normp['heat_value'].fillna(0, inplace=True)
dfu = pd.pivot(df_res_normp, index='pheno', columns='gene', values='heat_value')

In [7]:
df_ivw['heat_value'] = df_ivw['log_fdrp'] * df_ivw['dir']
df_res_fdrp = pd.merge(df_res, df_ivw[['id.exposure', 'id.outcome', 'heat_value']], left_on=['gene', 'pheno'],
                  right_on=['id.exposure', 'id.outcome'], how='left')
df_res_fdrp['heat_value'].fillna(0, inplace=True)
dfu2 = pd.pivot(df_res_fdrp, index='pheno', columns='gene', values='heat_value')

In [8]:
x = np.array(dfu)
dfu.iloc[:, :] = x
dfu.to_csv(f'results_merged/heatmap_{pheno}_{bag_type}.csv')

In [9]:
x2 = np.array(dfu2)
dfu2.iloc[:, :] = x2
# dfu2.to_csv(f'results_merged/heatmap_{pheno}_{bag_type}.csv')
# print(len(pd.unique(dfu['ensembl_gene_id'])))

In [10]:
p1 = -np.log10(0.05)
p2 = -np.log10(0.05 / len(df_phenos))

In [11]:
annot = np.zeros_like(x, dtype=str)
annot[np.where(abs(x) > p1)] = '◇'
annot[np.where(abs(x2) > p1)] = '★'

In [12]:
df_annot = pd.DataFrame(annot)
df_annot.index = dfu.index
df_annot.columns = dfu.columns

In [13]:
df_annot.to_csv(f'results_merged/heatmap_{pheno}_{bag_type}_annot.csv')